# Attempt 1: Technically Incorrect but Surprisingly Low RMS Log Error

## idea:
* Drop columns that seems useless from quick data exploration
* Numericalize the rest using integer code
* Add `Age` column == sale date - year made
* Train six models, one for each equipment group

## feature matrix:
(see `df_cleaner()` function)

<br>
<hr style='	background-color: #fff; border-top: 2px dashed #8c8b8b;'>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.base import clone
from sklearn.preprocessing import MinMaxScaler
import zipfile

In [2]:
def df_cleaner(df):

    
    df = df.copy()
    
    
    ## Han
    colnames = ['ProductSize','state', 'ProductGroup', 'Drive_System',\
                'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control', 'Stick','Transmission']

    df.drop(['MachineID',
             'fiModelDescriptor', 
             'fiProductClassDesc', 
             'ProductGroupDesc'], axis=1, inplace=True)
    
    df.drop(df[['SalesID',
                'fiModelDesc',
                'fiSecondaryDesc', 
                'fiModelSeries']], axis=1, inplace=True)
    
    df.drop(['Blade_Extension', 
             'Blade_Width', 
             'Pushblock', 
             'Scarifier', 
             'Tire_Size', 
             'Coupler_System'], axis=1, inplace=True)
  
    
    # temporary drop
    df.drop('fiBaseModel', axis=1, inplace=True)

    
    
    for colname in colnames:
        df[colname] = pd.Categorical(df[colname]).codes + 2
            
            
            
    ## Lily        
    for cname in df.columns[-13:]:
        df[cname] = pd.Categorical(df[cname]).codes + 2

        
        
                
    ## PR            
    column_names = ['Turbocharged','Hydraulics','Enclosure_Type','Engine_Horsepower','Ripper','Tip_Control','Coupler' ]
    for i,names in enumerate(column_names):
        unique_vals = df[names].unique()
        for n, vals in enumerate(unique_vals):
            if type(vals) is str:
                df.loc[df[names] == vals, names] = n
            else:
                df.loc[df[names].isnull(), names] = n

    
    ## Engel
    df["UsageBand"] = df["UsageBand"].fillna("Medium")
    df.loc[df["UsageBand"] == "Low", "UsageBand"] = 1
    df.loc[df["UsageBand"] == "Medium", "UsageBand"] = 2
    df.loc[df["UsageBand"] == "High", "UsageBand"] = 3
    df["auctioneerID"] = df["auctioneerID"].fillna(df["auctioneerID"].median())

    df["MachineHoursCurrentMeter"] = df["MachineHoursCurrentMeter"].fillna(df["MachineHoursCurrentMeter"].median())
     
    df.drop([ \
#              'Hydraulics_Flow',
#              'Blade_Type',
#              'Stick_Length',
#              'Ripper',
#              'Coupler',
#              'Undercarriage_Pad_Width',
#              'Thumb',
#              'UsageBand',
#              'Transmission',
#              'Ride_Control',
#              'Grouser_Type',
#              'Tip_Control',
#              'Drive_System',
#              'Travel_Controls',
#              'Forks',
#              'Enclosure_Type',
#              'Differential_Type',
#              'Steering_Controls',
#              'Pattern_Changer',
#              'Engine_Horsepower',
#              'Track_Type',
#              'Stick',
#              'Backhoe_Mounting',
#              'Pad_Type',
#              'Turbocharged',
#              'Grouser_Tracks' \
             ], axis=1, inplace=True)

    df['time_manf_sale'] = pd.DatetimeIndex(df['saledate']).year - df['YearMade']
#     df['AgeSize'] = df.time_manf_sale * df.ProductSize
#     df['GroupSize'] = df.ProductGroup * df.ProductSize
    
    return df

In [3]:
def train_models(df, base_estimator, feat_import=None):
    
    models = {}
    grpdf = df.groupby('ProductGroup')
    for key, grp in grpdf:
        print 'Training', key
        df2 = df[df['ProductGroup'] == key]
        df2 = df_cleaner(df2)
        
        y = df2['SalePrice'].values
        X = df2.drop(['SalePrice', 'saledate', 'ProductGroup'], axis=1).values
        
        
        if feat_import:
            X = X[:,feat_import[key]]
            
        estimator = clone(base_estimator)
        models[key] = estimator.fit(X, y)

    return models

In [4]:
def models_predict(models, df, y_available=True, feat_import=None):

    df.index = range(len(df))
    y_final = np.zeros(len(df))
    
    grpdf = df.groupby('ProductGroup')
    for key, grp in grpdf:
        print 'Testing', key
        
        inds = df['ProductGroup'] == key
        df2 = df[inds]
        df2 = df_cleaner(df2)
        
        if y_available:
            y = df2['SalePrice'].values
            X = df2.drop(['SalePrice', 'saledate', 'ProductGroup'], axis=1).values
        else:
            X = df2.drop(['saledate', 'ProductGroup'], axis=1).values
        
        
        if feat_import:
            X = X[:,feat_import[key]]
        
        
        y_pred = models[key].predict(X)
        y_final[list(df[inds].index)] += y_pred
    
    if y_available:
        print r2_score(df['SalePrice'].values, y_final)

    return y_final

In [8]:
df_train = pd.read_csv(zipfile.ZipFile('data/Train.zip', mode='r').open('Train.csv'))
df_test = pd.read_csv(zipfile.ZipFile('data/Test.zip', mode='r').open('test.csv'))

# df_train = pd.read_csv('data/Train.csv')#, parse_dates=['saledate'])
# df_test = pd.read_csv('data/test.csv')#, parse_dates=['saledate'])

/Users/l/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
models = train_models(df_train.copy(), RandomForestRegressor(50))

Training BL
Training MG
Training SSL
Training TEX
Training TTT
Training WL


In [10]:
y_pred = models_predict(models, df_test.copy(),y_available=False)

Testing BL
Testing MG
Testing SSL
Testing TEX
Testing TTT
Testing WL


In [11]:
result_df = pd.DataFrame(np.c_[df_test['SalesID'].values, y_pred], columns = ['SalesID', 'SalePrice'])
result_df.to_csv('result_group1.csv', index=False)

In [13]:
# python score_model.py result.csv

# Quick Test -- Run this first!

In [94]:
# Quick Test

df = df_train
np.random.seed(42)

rand_i = range(len(df))
np.random.shuffle(rand_i)
rand_i = rand_i[:2000]
test_size = int(len(rand_i)*0.2)

sub_df_test = df.iloc[rand_i[:test_size], :]
sub_df_train = df.iloc[rand_i[test_size:], :]

models = train_models(sub_df_train.copy(), RandomForestRegressor(50))
y_pred = models_predict(models, sub_df_test.copy())

Training BL
Training MG
Training SSL
Training TEX
Training TTT
Training WL
Testing BL
Testing MG
Testing SSL
Testing TEX
Testing TTT
Testing WL
0.713170483446
